## Imports

In [28]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [29]:
df = pd.read_csv('redfin_2023-08-01-11-27-31.csv')

## Data Clean

In [30]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [31]:
df = df.dropna(subset=['SOLD DATE'])

In [32]:
# Define list of desired months (excluding current month)
desired_months = ['July']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [33]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    1393
Name: count, dtype: int64
-------
$/SQUARE FEET
False    1378
True       15
Name: count, dtype: int64
-------
YEAR BUILT
False    1393
Name: count, dtype: int64
-------


In [34]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [35]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [36]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
312,PAST SALE,July-25-2023,Condo/Co-op,9917 W Okeechobee Rd Unit 4-305,Hialeah Gardens,FL,33016.0,250.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/9917-W-Okeec...,MARMLS,A11379050,N,Y,25.866333,-80.334622
760,PAST SALE,July-5-2023,Condo/Co-op,14951 Royal Oaks Ln #2306,North Miami,FL,33181.0,400.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/14951-Ro...,MARMLS,A11365864,N,Y,25.913867,-80.151668
1377,PAST SALE,July-20-2023,Condo/Co-op,501 NE 31st St #2610,Miami,FL,33137.0,710.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/501-NE-31st-St...,MARMLS,A11232266,N,Y,25.806882,-80.187314
1115,PAST SALE,July-18-2023,Condo/Co-op,290 174th St #308,Sunny Isles Beach,FL,33160.0,44500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/29...,MARMLS,A11332103,N,Y,25.936045,-80.130062
1152,PAST SALE,July-26-2023,Condo/Co-op,20000 E Country Club Dr #409,Aventura,FL,33180.0,51000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20000-E-Cou...,MARMLS,A11358484,N,Y,25.962368,-80.127549
1166,PAST SALE,July-21-2023,Condo/Co-op,13480 NE 6th Ave #107,North Miami,FL,33161.0,67000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11332382,N,Y,25.898940,-80.187168
1357,PAST SALE,July-26-2023,Condo/Co-op,13480 NE 6th Ave #216,North Miami,FL,33161.0,69885.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11267636,N,Y,25.898940,-80.187168
423,PAST SALE,July-14-2023,Condo/Co-op,8851 Sunrise Lakes Blvd #112,Sunrise,FL,33322.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/8851-Sunrise...,Beaches MLS,F10384679,N,Y,26.163868,-80.269112
1202,PAST SALE,July-14-2023,Condo/Co-op,8120 SW 24th St #109,North Lauderdale,FL,33068.0,78000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Lauderdale/812...,Beaches MLS,F10366840,N,Y,26.198492,-80.236448
285,PAST SALE,July-14-2023,Condo/Co-op,2751 E Sunrise Lakes Dr #306,Sunrise,FL,33322.0,80500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2751-E-Sunri...,MARMLS,A11398442,N,Y,26.159563,-80.254296


In [37]:
print(df_filtered['URL'].iloc[382])

https://www.redfin.com/FL/Oakland-Park/3020-NE-16th-Ave-33334/unit-201/home/41676923


In [38]:
# Correct the prices, if needed
df_filtered.at[312,'PRICE']=(250000)
df_filtered.at[760,'PRICE']=(400000)
df_filtered.at[1377,'PRICE']=(710000)
df_filtered.at[1115,'PRICE']=(445000)

In [39]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [40]:
# # Corrections, if needed
df_filtered.at[312,'$/SQUARE FEET']=(250000/1190)
df_filtered.at[760,'$/SQUARE FEET']=(400000/1730)
df_filtered.at[1377,'$/SQUARE FEET']=(710000/1055)
df_filtered.at[1115,'$/SQUARE FEET']=(445000/850)

In [41]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
1152,51000.0,20000 E Country Club Dr #409,Aventura,41.0
1357,69885.0,13480 NE 6th Ave #216,North Miami,82.0
1202,78000.0,8120 SW 24th St #109,North Lauderdale,92.0
1166,67000.0,13480 NE 6th Ave #107,North Miami,92.0
794,103000.0,4401 NW 41st St #307,Lauderdale Lakes,93.0
1002,87000.0,4851 NW 21st St #102,Lauderhill,94.0
1212,90000.0,4740 NW 21st St #408,Lauderhill,94.0
1349,90000.0,5001 NW 34th St #203,Lauderdale Lakes,95.0
1195,90500.0,4770 NW 21st St #203,Lauderhill,98.0
735,93000.0,3776 Inverrary Blvd Unit 207R,Lauderhill,103.0


In [42]:
print(df_filtered.URL.iloc[760])

https://www.redfin.com/FL/North-Miami/14951-Royal-Oaks-Ln-33181/unit-2306/home/43397600


In [43]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [44]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [45]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [46]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [47]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [48]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [49]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"July 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [50]:
m.save('index.html')

## Summary Info

In [51]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [52]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [53]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [54]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami/227-NE-2nd-St-33132/unit-2706/home/179812721


In [55]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
7400 OCEANSIDE AT FISHER, 7463 Fisher Island Dr #7463 Miami Beach | Price $24,175,000 | $3,177 psf | Year built: 2002
Least Expensive
TURNBERRY VLG NO TOWER CO, 20000 E Country Club Dr #409 Aventura | Price $51,000 | $41 psf | Year built: 2006
Highest Price Per Square Foot
7400 OCEANSIDE AT FISHER, 7463 Fisher Island Dr #7463 Miami Beach | Price $24,175,000 | $3,177 psf | Year built: 2002
Lowest Price Per Square Foot
TURNBERRY VLG NO TOWER CO, 20000 E Country Club Dr #409 Aventura | Price $51,000 | $41 psf | Year built: 2006
Newest
YOTEL PAD, 227 NE 2nd St #2706 Miami | Price $570,000 | $805 psf | Year built: 2022
Oldest
PALM GARDEN CONDO, 760 Meridian Ave #1 Miami Beach | Price $735,000 | $721 psf | Year built: 1923


## Time on Market Calculator

In [56]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [57]:
from datetime import datetime, timedelta

date1 = datetime(2023, 3, 1) ## List (Earlier) date
date2 = datetime(2023, 6, 29) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

120


## Map URL Snagger

In [58]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [59]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_july_2023


## Get Summary Data

In [60]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 1393
--------
Total sale price: $767,033,014
--------
Median sale price: $325,000
--------
Max sale price: $24,175,000
--------
Min sale price: $51,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,177
--------
Min price per square foot: $41
--------
Median price per square foot: $336
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1923.0
----------
Average building age: 1985.2778176597271
